⊕ [structured-streaming-python.html - Databricks](https://docs.databricks.com/_static/notebooks/structured-streaming-python.html)


In [3]:
import os
print(os.getcwd())

/Users/xiaofeiwu/jcloud/assets/langs/workspace/spark/workspace/notebook


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [9]:
from pyspark.sql.types import *

inputPath = "./events/"

# Since we know the data format already, let's define the schema to speed up processing (no need for Spark to infer schema)
jsonSchema = StructType([ StructField("time", TimestampType(), True), StructField("action", StringType(), True) ])
# Static DataFrame representing data in the JSON files
staticInputDF = (
  spark
    .read
    .schema(jsonSchema)
    .json(inputPath)
)
display(staticInputDF)
staticInputDF.show()

DataFrame[time: timestamp, action: string]

+-------------------+------+
|               time|action|
+-------------------+------+
|2016-07-26 10:45:07|  Open|
|2016-07-26 10:45:47|  Open|
|2016-07-26 10:46:42|  Open|
|2016-07-26 10:46:59|  Open|
|2016-07-26 10:47:05|  Open|
|2016-07-26 10:47:14|  Open|
|2016-07-26 10:47:25|  Open|
|2016-07-26 10:47:26|  Open|
|2016-07-26 10:47:28|  Open|
|2016-07-26 10:47:36|  Open|
|2016-07-26 10:47:44|  Open|
|2016-07-26 10:47:46|  Open|
|2016-07-26 10:47:47|  Open|
|2016-07-26 10:47:49|  Open|
|2016-07-26 10:47:51|  Open|
|2016-07-26 10:48:02|  Open|
|2016-07-26 10:48:05|  Open|
|2016-07-26 10:48:11|  Open|
|2016-07-26 10:48:17|  Open|
|2016-07-26 10:48:23|  Open|
+-------------------+------+
only showing top 20 rows



Now we can compute the number of "open" and "close" actions with one hour windows. To do this, we will group by the action column and 1 hour windows over the time column.



In [10]:
from pyspark.sql.functions import *      # for window() function

staticCountsDF = (
  staticInputDF
    .groupBy(
       staticInputDF.action, 
       window(staticInputDF.time, "1 hour"))    
    .count()
)
staticCountsDF.cache()

# Register the DataFrame as table 'static_counts'
staticCountsDF.createOrReplaceTempView("static_counts")


In [14]:
# %sql select action, sum(count) as total_count from static_counts group by action
df=spark.sql("select action, sum(count) as total_count from static_counts group by action")
df.show()

+------+-----------+
|action|total_count|
+------+-----------+
|  Open|        184|
| Close|         12|
+------+-----------+



In [15]:
# How about a timeline of windowed counts?
str='''select action, date_format(window.end, "MMM-dd HH:mm") as time, count from static_counts order by time, action'''
df=spark.sql(str)
df.show()

+------+------------+-----+
|action|        time|count|
+------+------------+-----+
| Close|Jul-26 11:00|   11|
|  Open|Jul-26 11:00|  179|
| Close|Jul-26 12:00|    1|
|  Open|Jul-26 12:00|    5|
+------+------------+-----+



## Stream Processing
Now that we have analyzed the data interactively, let's convert this to a streaming query that continuously updates as data comes. Since we just have a static set of files, we are going to emulate a stream from them by reading one file at a time, in the chronological order they were created. The query we have to write is pretty much the same as the interactive query above.

In [16]:
from pyspark.sql.functions import *

# Similar to definition of staticInputDF above, just using `readStream` instead of `read`
streamingInputDF = (
  spark
    .readStream                       
    .schema(jsonSchema)               # Set the schema of the JSON data
    .option("maxFilesPerTrigger", 1)  # Treat a sequence of files as a stream by picking one file at a time
    .json(inputPath)
)

# Same query as staticInputDF
streamingCountsDF = (                 
  streamingInputDF
    .groupBy(
      streamingInputDF.action, 
      window(streamingInputDF.time, "1 hour"))
    .count()
)

# Is this DF actually a streaming DF?
streamingCountsDF.isStreaming

True

As you can see, streamingCountsDF is a streaming Dataframe (streamingCountsDF.isStreaming was true). You can start streaming computation, by defining the sink and starting it. In our case, we want to interactively query the counts (same queries as above), so we will set the complete set of 1 hour counts to be in a in-memory table (note that this for testing purpose only in Spark 2.0).



In [17]:
spark.conf.set("spark.sql.shuffle.partitions", "2")  # keep the size of shuffles small

query = (
  streamingCountsDF
    .writeStream
    .format("memory")        # memory = store in-memory table (for testing only in Spark 2.0)
    .queryName("counts")     # counts = name of the in-memory table
    .outputMode("complete")  # complete = all the counts should be in the table
    .start()
)

query is a handle to the streaming query that is running in the background. This query is continuously picking up files and updating the windowed counts.

Note the status of query in the above cell. The progress bar shows that the query is active. Furthermore, if you expand the > counts above, you will find the number of files they have already processed.

Let's wait a bit for a few files to be processed and then interactively query the in-memory counts table.

In [18]:
from time import sleep
sleep(5)  # wait a bit for computation to start

In [20]:
df=spark.sql('select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action')
df.show()

+------+------------+-----+
|action|        time|count|
+------+------------+-----+
| Close|Jul-26 11:00|   11|
|  Open|Jul-26 11:00|  179|
| Close|Jul-26 12:00|   59|
|  Open|Jul-26 12:00|  188|
+------+------------+-----+



In [21]:
sleep(5)

df=spark.sql('select action, date_format(window.end, "MMM-dd HH:mm") as time, count from counts order by time, action')
df.show()

+------+------------+-----+
|action|        time|count|
+------+------------+-----+
| Close|Jul-26 11:00|   11|
|  Open|Jul-26 11:00|  179|
| Close|Jul-26 12:00|   59|
|  Open|Jul-26 12:00|  188|
+------+------------+-----+



In [22]:
sqlstr='''select action, sum(count) as total_count from counts group by action order by action'''
df=spark.sql(sqlstr)
df.show()

+------+-----------+
|action|total_count|
+------+-----------+
| Close|         70|
|  Open|        367|
+------+-----------+



If you keep running the above query repeatedly, you will always find that the number of "opens" is more than the number of "closes", as expected in a data stream where a "close" always appear after corresponding "open". This shows that Structured Streaming ensures prefix integrity. Read the blog posts linked below if you want to know more.

Note that there are only a few files, so consuming all of them there will be no updates to the counts. Rerun the query if you want to interact with the streaming query again.

Finally, you can stop the query running in the background, either by clicking on the 'Cancel' link in the cell of the query, or by executing query.stop(). Either way, when the query is stopped, the status of the corresponding cell above will automatically update to TERMINATED.

In [23]:
query.stop()